In [8]:
import pandas as pd

# Load the Excel file
file_path = '/Users/faizantahir/Documents/LUT uni/Term 2/Product Analytics/Module 5/Assignment/Telepass.xlsx'
xls = pd.ExcelFile(file_path)

# Get the names of all the sheets in the Excel file
sheet_names = xls.sheet_names

# Dictionary to hold data from each sheet
sheets_data = {}

# Iterate over each sheet name and read the content
for sheet in sheet_names:
    sheets_data[sheet] = pd.read_excel(xls, sheet_name=sheet)

# Now let's output a summary of each sheet: its name, the first few rows, and some statistics
sheets_summary = {sheet: {'preview': data.head(), 'description': data.describe(include='all')}
                  for sheet, data in sheets_data.items()}

sheets_summary



{'Copyright': {'preview':          Telepass: From Tolling to Mobility Platform
  0               Harvard Business School Case 622-011
  1                                Courseware: 622-701
  2  This courseware was prepared solely as the bas...
  3                                                NaN
  4  This spreadsheet supplements "Telepass: From T...,
  'description':        Telepass: From Tolling to Mobility Platform
  count                                            7
  unique                                           7
  top           Harvard Business School Case 622-011
  freq                                             1},
 'Data Dictionary': {'preview':           Data Dictionary Unnamed: 1  \
  0  "Insurance Quotes" Tab        NaN   
  1                Variable       Type   
  2               client_id    Numeric   
  3            quotation_id    Numeric   
  4            driving_type  Character   
  
                                            Unnamed: 2  
  0                  

In [9]:
# Load the specific sheets 'Insurance Quotes' and 'Transactions' from the Exce

insurance_quotes_df = pd.read_excel(file_path, sheet_name='Insurance Quotes')
transactions_df = pd.read_excel(file_path, sheet_name='Transactions')

# Display the first few rows of each sheet
print("Insurance Quotes Sheet:")
print(insurance_quotes_df.head())
print("\nTransactions Sheet:")
print(transactions_df.head())

# Get a concise summary of each sheet
print("\nInsurance Quotes Info:")
insurance_quotes_df.info()
print("\nTransactions Info:")
transactions_df.info()

# Check for missing values in each sheet
print("\nMissing values in Insurance Quotes:")
print(insurance_quotes_df.isnull().sum())
print("\nMissing values in Transactions:")
print(transactions_df.isnull().sum())

# You may also want to describe the data to see statistical summaries for numeric columns
print("\nDescription of Insurance Quotes:")
print(insurance_quotes_df.describe())
print("\nDescription of Transactions:")
print(transactions_df.describe())



Insurance Quotes Sheet:
   client_id  quotation_id  driving_type car_immatriculation_date  \
0      23789          3649  more_than_26               2011-07-20   
1      10133         28552  more_than_26               2007-03-01   
2      20785         31958  more_than_26               2017-08-30   
3      33892         27614  more_than_26               2017-07-25   
4      34153          1529  more_than_26               2006-12-19   

          car_brand       car_model insurance_expires_at           birth_date  \
0              FORD           C-MAX           2020-06-12                  NaN   
1  FIAT - INNOCENTI      FIAT CROMA           2020-01-22                  NaN   
2            NISSAN  NISSAN QASHQAI           2020-10-02  1992-09-22 00:00:00   
3  FIAT - INNOCENTI      FIAT PANDA           2020-07-25  1996-02-09 00:00:00   
4  FIAT - INNOCENTI      FIAT PUNTO           2020-10-13  2000-01-08 00:00:00   

  gender county  ... natural_events theft_fire kasko license_revoked  \
0 

In [10]:
# Merge the dataframes on 'client_id' for an inner join
linked_data = pd.merge(
    left=insurance_quotes_df, 
    right=transactions_df, 
    how='inner', 
    on='client_id'
)


print(linked_data.head())


linked_data.to_excel('/Users/faizantahir/Documents/LUT uni/Term 2/Product Analytics/Module 5/Assignment/linked_data.xlsx', index=False)


   client_id  quotation_id       driving_type car_immatriculation_date  \
0      20785         31958       more_than_26               2017-08-30   
1      20785         31958       more_than_26               2017-08-30   
2      34153          1529       more_than_26               2006-12-19   
3      21141         10960  between_23_and_25               2009-07-20   
4      21141         10960  between_23_and_25               2009-07-20   

          car_brand       car_model insurance_expires_at           birth_date  \
0            NISSAN  NISSAN QASHQAI           2020-10-02  1992-09-22 00:00:00   
1            NISSAN  NISSAN QASHQAI           2020-10-02  1992-09-22 00:00:00   
2  FIAT - INNOCENTI      FIAT PUNTO           2020-10-13  2000-01-08 00:00:00   
3         CHEVROLET            AVEO           2020-01-23                  NaN   
4         CHEVROLET            AVEO           2020-01-23                  NaN   

  gender county  ... vandalism key_loss price_sale price_full  \
0  

In [11]:


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

# Load the data

df = pd.read_excel('/Users/faizantahir/Documents/LUT uni/Term 2/Product Analytics/Module 5/Assignment/linked_data.xlsx')
# Preprocessing
# Convert dates to datetime and calculate the duration where applicable
date_cols = ['car_immatriculation_date', 'insurance_expires_at', 'birth_date',
             'base_subscription', 'pay_subscription', 'pay_cancellation',
             'premium_subscription', 'premium_cancellation', 'policy_quoted_at','year_month']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Calculate age of the car
df['car_age'] = (datetime.now() - df['car_immatriculation_date']).dt.days // 365

# Drop the original date columns as we will use processed features instead
df.drop(columns=date_cols, inplace=True)

# Handle categorical variables with Label Encoding
categorical_cols = ['driving_type', 'car_brand', 'car_model', 'gender', 'county', 'base_type',
                    'operating_system', 'guarantees_purchased', 'guarantees_available']
label_encoders = {}
for col in categorical_cols:
    label_encoder = LabelEncoder()
    df[col] = label_encoder.fit_transform(df[col].astype(str))
    label_encoders[col] = label_encoder

# Handle missing values - fill with median for numeric and mode for categorical
for col in df.columns:
    if df[col].dtype == 'object':
        df[col].fillna(df[col].mode()[0], inplace=True)
    else:
        df[col].fillna(df[col].median(), inplace=True)

   
try:
  # Try converting to integer directly (assuming TRUE/FALSE are the only values)
  df['issued'] = df['issued'].astype(bool)
except ValueError:
  # If conversion fails, replace non-numeric values with NaN
  df['issued'] = df['issued'].replace({'TRUE': 1, 'FALSE': 0}, regex=True).astype(bool)



# Confirm that we have no NaN values in the 'issued' column
assert df['issued'].isna().sum() == 0, "NaN values present in 'issued' column"

# Confirm that the DataFrame is not empty
assert not df.empty, "DataFrame is empty"

df = df.dropna(subset=['issued'])


for column in df.columns:
    if df[column].dtype in ['int64', 'float64']:
        df[column].fillna(df[column].median(), inplace=True)
    else:
        # For categorical features, we can fill NaN values with the mode (most common value)
        # or drop the feature if too many are missing
        df[column] = df[column].fillna(df[column].mode().iloc[0])

# Encode categorical features using Label Encoding or One-Hot Encoding
# For simplicity, let's apply Label Encoding
categorical_features = df.select_dtypes(include=['object', 'bool']).columns
label_encoder = LabelEncoder()
for feature in categorical_features:
    df[feature] = label_encoder.fit_transform(df[feature])




df = df.dropna(subset=['issued'])

# Since the model cannot handle NaN or missing values, we need to fill or impute these
# Let's impute missing numeric data with median and categorical data with mode
for column in df.columns:
    if df[column].dtype in ['int64', 'float64']:
        df[column].fillna(df[column].median(), inplace=True)
    else:
        # For categorical features, we can fill NaN values with the mode (most common value)
        # or drop the feature if too many are missing
        df[column] = df[column].fillna(df[column].mode().iloc[0])

# Encode categorical features using Label Encoding or One-Hot Encoding
# For simplicity, let's apply Label Encoding
categorical_features = df.select_dtypes(include=['object', 'bool']).columns
label_encoder = LabelEncoder()
for feature in categorical_features:
    df[feature] = label_encoder.fit_transform(df[feature])




# Define features and target variable
X = df.drop(['client_id', 'quotation_id', 'issued', 'car_brand', 'car_model', 'gender', 'county', 'operating_system'], axis=1)
y = df['issued']
#print(" Y :", y) 

pd.set_option('display.max_columns', None)

# Print info of DataFrame
print(df.info())

# Reset display options to default
pd.reset_option('display.max_columns')



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49362 entries, 0 to 49361
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   client_id                 49362 non-null  int64  
 1   quotation_id              49362 non-null  int64  
 2   driving_type              49362 non-null  int64  
 3   car_brand                 49362 non-null  int64  
 4   car_model                 49362 non-null  int64  
 5   gender                    49362 non-null  int64  
 6   county                    49362 non-null  int64  
 7   base_type                 49362 non-null  int64  
 8   operating_system          49362 non-null  int64  
 9   broker_id                 49362 non-null  int64  
 10  issued                    49362 non-null  int64  
 11  guarantees_purchased      49362 non-null  int64  
 12  guarantees_available      49362 non-null  int64  
 13  roadside_assistance       49362 non-null  int64  
 14  driver

/var/folders/y9/4pnmdh21585406rhq_g74jbc0000gn/T/ipykernel_20458/3849111333.py:39: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/var/folders/y9/4pnmdh21585406rhq_g74jbc0000gn/T/ipykernel_20458/3849111333.py:39: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa

In [12]:

X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.2, train_size=0.8 ,random_state=42)


from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming X_train, y_train, X_test, and y_test are already defined and preprocessed.

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier on the training data
rf_classifier.fit(X_train, y_train)

# Predict on the test data
y_pred_rf = rf_classifier.predict(X_test)

# Calculate accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)

# Generate classification report (optional, but recommended)
class_report_rf = classification_report(y_test, y_pred_rf)

# Output the results
print("Random Forest Accuracy:", accuracy_rf)
print("Classification Report:")
print(class_report_rf)




Random Forest Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7028
           1       1.00      1.00      1.00      2845

    accuracy                           1.00      9873
   macro avg       1.00      1.00      1.00      9873
weighted avg       1.00      1.00      1.00      9873



In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming X_train, y_train, X_test, and y_test are already defined and preprocessed.
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.2, train_size=0.8 ,random_state=42)
# Initialize the Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Train the classifier on the training data
dt_classifier.fit(X_train, y_train)

# Predict on the test data
y_pred_dt = dt_classifier.predict(X_test)

# Calculate accuracy
accuracy_dt = accuracy_score(y_test, y_pred_dt)

# Generate classification report (optional, but recommended)
class_report_dt = classification_report(y_test, y_pred_dt)

# Output the results
print("Decision Tree Accuracy:", accuracy_dt)
print("Classification Report:")
print(class_report_dt)


Decision Tree Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7028
           1       1.00      1.00      1.00      2845

    accuracy                           1.00      9873
   macro avg       1.00      1.00      1.00      9873
weighted avg       1.00      1.00      1.00      9873



In [15]:
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.2, train_size=0.8 ,random_state=42)


# Initialize the Logistic Regression Classifier
logistic_classifier = LogisticRegression(random_state=42)

# Train the classifier on the training data
logistic_classifier.fit(X_train, y_train)

# Predict on the test data
y_pred_logistic = logistic_classifier.predict(X_test)

# Calculate accuracy
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)

# Generate classification report (optional, but recommended)
class_report_logistic = classification_report(y_test, y_pred_logistic)

# Output the results
print("Logistic Regression Accuracy:", accuracy_logistic)
print("Classification Report:")
print(class_report_logistic)


Logistic Regression Accuracy: 0.8481717816266585
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.95      0.90      7028
           1       0.84      0.59      0.69      2845

    accuracy                           0.85      9873
   macro avg       0.84      0.77      0.80      9873
weighted avg       0.85      0.85      0.84      9873



/Users/faizantahir/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
